# Comparison between ANN and LSTM result for Air Passengers Dataset using Tensorflow


Inspired by 

https://youtu.be/tKM5d8Ll1k0

https://github.com/bnsreenu/python_for_microscopists.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
path = '../input/air-passengers/AirPassengers.csv'
df = pd.read_csv(path)
print(df.dtypes)
print(df.head())

In [ ]:
df['Month'] = pd.to_datetime(df['Month'])
print(df.dtypes)
print(df.head())

In [ ]:
df.set_index('Month', inplace=True) 
print(df.tail())
plt.plot(df['#Passengers'])

In [ ]:
df['year'] = [d.year for d in df.index]
print(df.tail())
df['month'] = [d.strftime('%b') for d in df.index]
print(df.tail())
years = df['year'].unique()
print(years)

In [ ]:
#Plot yearly and monthly values as boxplot
sns.boxplot(x='year', y='#Passengers', data=df)
plt.figure()
sns.boxplot(x='month', y='#Passengers', data=df)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose 
decomposed = seasonal_decompose(df['#Passengers'],model ='additive')

In [ ]:
trend = decomposed.trend
seasonal = decomposed.seasonal #Cyclic behavior may not be seasonal!
residual = decomposed.resid

plt.figure(figsize=(12,8))
plt.subplot(411)
plt.plot(df['#Passengers'], label='Original', color='red')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend, label='Trend', color='red')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color='red')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(residual, label='Residual', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
dataset = df['#Passengers'].values
dataset = dataset.astype('float32')
dataset = np.expand_dims(dataset,axis=-1)
print(dataset.shape)

In [ ]:
plt.plot(dataset)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
plt.plot(dataset)

In [ ]:
train_size = int(len(dataset) * 0.66)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(dataset), train_size, test_size)

In [ ]:
def to_sequences(dataset, seq_size):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
#         print(i)
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])
        
    return np.array(x),np.array(y)

In [ ]:
seq_size = 20 # Number of time steps to look back 
trainX, trainY = to_sequences(train, seq_size);
testX, testY = to_sequences(test, seq_size);

In [ ]:
print("Shape of training set:", trainX.shape, trainY.shape)
print("Shape of test set: ", testX.shape, testY.shape)
print(trainX[11], trainY[11])

In [ ]:
import tensorflow as tf
import tensorflow.keras

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim=seq_size, activation='relu'), #12
    tf.keras.layers.Dense(32, activation='relu'), #8
    tf.keras.layers.Dense(1),
])
model.compile(loss='mean_squared_error', optimizer='adam')
print(model.summary()) 
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.fit(trainX, trainY, validation_data=(testX, testY),verbose=2, epochs=100)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict) # predicted 
trainY_inverse = scaler.inverse_transform([trainY]) # original
testPredict = scaler.inverse_transform(testPredict)
testY_inverse = scaler.inverse_transform([testY])

In [ ]:
import math
trainScore = math.sqrt(mean_squared_error(trainY_inverse[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY_inverse[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[seq_size:len(trainPredict)+seq_size, :] = trainPredict   # answer will start from seq_size

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(seq_size*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.figure(figsize = (16,16))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
from keras.layers import LSTM, Flatten

In [ ]:
# Reshape input to be [samples, time steps, features]
seq_size = 20 # Number of time steps to look back 
trainX, trainY = to_sequences(train, seq_size);
testX, testY = to_sequences(test, seq_size);

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
print(trainX.shape)

In [ ]:
# Single LSTM with hidden Dense
model = Sequential()
model.add(LSTM(16, activation='relu', input_shape=(None, seq_size)))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

print(model.summary()) 
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.fit(trainX, trainY, validation_data=(testX, testY), verbose=2, epochs=100)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# shift train predictions for plotting
#we must shift the predictions so that they align on the x-axis with the original dataset. 
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[seq_size:len(trainPredict)+seq_size, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(seq_size*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
plt.figure(figsize = (12,12))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()